# Figure 1

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from ngmt.datasets import mobilised
from ngmt.modules.gsd import ParaschivIonescuGaitSequenceDetection
from ngmt.modules.icd import ParaschivIonescuInitialContactDetection
from ngmt.config import cfg_colors

# Read the data
file_path = (
    r"C:\Users\Project\Desktop\Gait_Sequence\Mobilise-D dataset_1-18-2023\CHF\data.mat"
)

# Define tracking system and tracked points
tracking_sys = "SU"
tracked_points = {tracking_sys: ["LowerBack"]}

# Load recording data
recording = mobilised.load_recording(
    file_name=file_path, tracking_systems=[tracking_sys], tracked_points=tracked_points
)

# Extract lower back acceleration data
acceleration_data = recording.data[tracking_sys][
    ["LowerBack_ACCEL_x", "LowerBack_ACCEL_y", "LowerBack_ACCEL_z"]
]

# Get sampling frequency
sampling_frequency = recording.channels[tracking_sys][
    recording.channels[tracking_sys]["name"] == "LowerBack_ACCEL_x"
]["sampling_frequency"].values[0]

# Gait sequence detection
gsd = ParaschivIonescuGaitSequenceDetection(target_sampling_freq_Hz=40)
gsd = gsd.detect(
    data=acceleration_data, sampling_freq_Hz=sampling_frequency, plot_results=False
)
gait_sequences = gsd.gait_sequences_

# Initial contact detection
icd = ParaschivIonescuInitialContactDetection(target_sampling_freq_Hz=40)
icd = icd.detect(
    data=acceleration_data,
    gait_sequences=gait_sequences,
    sampling_freq_Hz=sampling_frequency,
)
initial_contacts = icd.initial_contacts_

# Access the first detected gait sequence
first_gait_sequence = gait_sequences.iloc[0]

# Plot setup
fig, ax = plt.subplots(figsize=(12, 8))
colors = cfg_colors["raw"]
num_samples = len(acceleration_data)
time_seconds = np.arange(num_samples) / sampling_frequency

# Initial contacts within the first gait sequence
ic_within_gait = initial_contacts[
    initial_contacts["onset"].between(
        first_gait_sequence["onset"],
        first_gait_sequence["onset"] + first_gait_sequence["duration"],
    )
]

# Plot raw acceleration data
for i in range(3):
    ax.plot(
        time_seconds,
        acceleration_data[f"LowerBack_ACCEL_{chr(120 + i)}"],
        color=colors[i],
        label=f"Acc {i + 1}",
    )

# Plot gait onset and duration
plt.axvline(first_gait_sequence["onset"], color="green", linestyle="-")
ax.axvspan(
    first_gait_sequence["onset"],
    first_gait_sequence["onset"] + first_gait_sequence["duration"],
    alpha=0.2,
    color="gray",
    label="Gait duration",
)

# Plot initial contacts within the first gait sequence
for ic_time in ic_within_gait["onset"]:
    ax.axvline(ic_time, color="blue", linestyle="--")

# Customize plot
start_limit = first_gait_sequence["onset"] - 1
end_limit = first_gait_sequence["onset"] + first_gait_sequence["duration"] + 1
ax.set_xlim(start_limit, end_limit)
ax.set_ylim(-1, 1.5)
ax.set_xlabel("Time (seconds)", fontsize=16)
ax.set_ylabel("Acceleration (g)", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(True, linestyle='--', linewidth=0.5)
ax.legend(fontsize=12)
plt.title("Acceleration Data and Gait Events", fontsize=18)
ax.legend(
    ["Acc 1", "Acc 2", "Acc 3", "Gait onset", "Gait duration", "Initial contacts"],
    fontsize=20,
    loc="upper right",
)
plt.tight_layout()

# Save the figure as a PNG
plt.savefig('figure_1.png', dpi=300, bbox_inches='tight')

# Save the figure as a PDF
plt.switch_backend('agg')
plt.savefig('figure_1.pdf', format='pdf', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


86 gait sequence(s) detected.


C:\Users\Project\AppData\Local\Temp\ipykernel_21540\3269734965.py:115: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


<Figure size 640x480 with 0 Axes>